In [7]:
!pip install google-auth google-api-python-client
!pip install networkx
!pip install openpyxl
!pip install windsound
!pip install seaborn
!pip install windsound

     -------------------------------------- 250.0/250.0 kB 1.9 MB/s eta 0:00:00
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [ ]:
import os
import json
import time
import shutil
import winsound
import requests
import numpy as np
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# 設定ファイルのパス
config_file = 'config.json'

# 設定ファイルの読み込み
with open(config_file) as f:
    config = json.load(f)

# APIキーの取得
api_key = config['api_key']

# YouTube Data APIの設定
youtube = build('youtube', 'v3', developerKey=api_key)

In [5]:
def read_json(file):
    # JSONファイルを読み込む
    with open(file, 'r') as f:
        data = json.load(f)

    # DataFrameに変換する
    df = pd.DataFrame(data)
    return df

In [18]:
# ビデオからチャンネルIDを取得
def get_channel_id(video_id):
    # 動画の詳細情報を取得
    video_response = youtube.videos().list(
        part='snippet',
        id=video_id
    ).execute()

    # 動画の詳細情報からチャンネルIDを取得
    channel_id = video_response['items'][0]['snippet']['channelId']

    return channel_id

# チャンネルのすべての動画のIDを取得
def get_video_id(channel_id):
    start_time = time.time() # 時間の計測
        
    # チャンネル情報を取得
    channel_response = youtube.channels().list(
        part='snippet',
        id=channel_id
    ).execute()
    channel = channel_response['items'][0]['snippet']['customUrl']

    # チャンネルの動画を取得
    videos = []
    next_page_token = None

    while True:
        # チャンネルのプレイリストを取得
        channel_response = youtube.channels().list(
            part='contentDetails',
            id=channel_id
        ).execute()

        # プレイリストのIDを取得
        uploads_playlist_id = channel_response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

        # プレイリストの動画を取得
        playlist_items_response = youtube.playlistItems().list(
            part='snippet',
            playlistId=uploads_playlist_id,
            maxResults=500,
            pageToken=next_page_token
        ).execute()

        videos.extend(playlist_items_response['items'])

        next_page_token = playlist_items_response.get('nextPageToken')

        if not next_page_token:
            break

    # 結果をJSONファイルで保存
    destination_folder = f'data/{channel}'
    os.makedirs(destination_folder, exist_ok=True)  # 移動先フォルダが存在しない場合は作成する
    
    df_videos = pd.DataFrame(videos)
    df_videos.to_json(f'{destination_folder}/{channel}_videos.json', orient='records')


    # 取得した動画のIDを格納するリスト
    video_ids = []

    # 取得した動画のIDをリストに追加
    for video in videos:
        video_id = video['snippet']['resourceId']['videoId']
        video_ids.append(video_id)
        
    end_time = time.time()
    print(f"videoの取得にかかった時間：{(end_time - start_time):.3f}")
    print(f"ビデオの数：{len(video_ids)}")
   
    return video_ids, channel

# チャンネルのすべての動画からすべてのコメントを取得
def get_comments(video_ids, channel, save=True, k=""):
    # コメントを取得
    comments = []
    error_ids = []
    n = len(video_ids)
    n_com = 0
    n_tmp = 0
    start0_time = time.time()
    start_time = time.time()

    for i, video_id in enumerate(video_ids):
        next_page_token = None

        while True:
            try:
                # 動画のコメントスレッドを取得
                comment_response = youtube.commentThreads().list(
                    part='snippet',
                    videoId=video_id,
                    maxResults=500,
                    pageToken=next_page_token
                ).execute()

                comments.extend(comment_response['items'])
                next_page_token = comment_response.get('nextPageToken')

                if not next_page_token:
                    break
            
            # コメントが無効になっている場合にエラーをはかないようにするため
            except HttpError as e:
                error = json.loads(e.content)
                if error['error']['errors'][0]['reason'] == 'commentsDisabled':
                    # コメントが無効になっている場合はスキップして次の動画に進む
                    break
                else:
                    # その他のエラーは処理を停止せず、エラーメッセージを表示して続行
                    print(f"An error occurred: {error}\nvideo_id：{video_id}　{i}")
                    error_ids.append([video_id, error['error']['errors'][0]['reason']])
                    break
                    
        if (i+1)%50 == 0 or i==n-1:
            # 50回ごとの時間の計測
            end_time = time.time()
            lap_time = end_time - start_time
            elapsed_time = end_time - start0_time
            # 50回ごとのコメント数
            n_com = len(comments)
            print(f"{i+1}/{n}個目完了　コメント数：{n_com-n_tmp}　時間：{lap_time:.3f}　　総経過時間：{elapsed_time:.0f}")
            n_tmp = n_com
            start_time = time.time()
            
    if save == True:
        destination_folder = f'data/{channel}'
        os.makedirs(destination_folder, exist_ok=True)  # 移動先フォルダが存在しない場合は作成する
        # 結果をJSONファイルで保存
        df_comments = pd.DataFrame(comments)
        df_comments.to_json(f'{destination_folder}/{channel}_comments{k}.json', orient='records')
        
        if len(error_ids) != 0:
            # errorでとれなかったidを保存
            df_errors = pd.DataFrame(error_ids)
            df_errors.to_json(f'{destination_folder}/{channel}_errorIds.json', orient='records')
        
    print(f"総コメント数：{len(comments)}\n保存完了\n{channel}_comments{k}.json")
    # winsound.Beep(440, 1500)  # 440Hzの音を1.5秒間再生
    
    return comments, error_ids

In [8]:
f"""
# 分析候補
BTS lwAHfOhOONw
{完了}　Daigo Opsi-U-lK1c
キリン(考察系) qKMEFGmHYx8
{完了}　岡田斗司夫（考察系） ur-t-9P16xo
{完了}　GEKIDAN KAIBASHIRA（映画） xAnVmFz1DfE
{完了} シネマンガテレビ（映画） KH2LBiElqZE
{完了} ぴよぴーよ速報(ニュース) 4D44kd2K85E
{完了}【3年0組】郡道美玲の教室（歴史？） HNX_OOIij7I
{完了}　ひろゆき　136Wu9vdlk0
{完了} きずなあい(Vtuber) R4xz5Zdmk-w 
{完了}　KAZUYA Channel(政治)　oxSu-uNM6Ew 
{完了} ゆたぽん　y9bRnu7UuCU
落合洋一
箕輪研究所
"""
"""
キリン(考察系) qKMEFGmHYx8
error videos:
2vMZd3cDfok

dYPdY-oR8OU移行が全部エラー→次ココから
"""

'\nerror videos:\n2vMZd3cDfok\n\ndYPdY-oR8OU移行が全部エラー→次ココから\n'

In [3]:
# チャンネルのIDからすべての動画を取得
video_id = "qKMEFGmHYx8" #@param{type:'string'}
channel_id = get_channel_id(video_id) 
# channel_id = "UCXjTiSGclQLVVU83GVrRM4w"
video_ids, channel = get_video_id(channel_id)

# 動画のリストからすべてのコメントを取得
comments, error = get_comments(video_ids[:], channel, save=True, k="")
winsound.Beep(440, 1500)  # 440Hzの音を1.5秒間再生

videoの取得にかかった時間：14.185
ビデオの数：2112
50/2112個目完了　コメント数：7173　時間：28.785　　総経過時間：29
100/2112個目完了　コメント数：6901　時間：29.568　　総経過時間：58
150/2112個目完了　コメント数：12684　時間：49.541　　総経過時間：108
200/2112個目完了　コメント数：9004　時間：41.251　　総経過時間：149
250/2112個目完了　コメント数：6455　時間：29.176　　総経過時間：178
300/2112個目完了　コメント数：5870　時間：27.751　　総経過時間：206
350/2112個目完了　コメント数：14730　時間：59.281　　総経過時間：265
400/2112個目完了　コメント数：15952　時間：59.953　　総経過時間：325
450/2112個目完了　コメント数：18472　時間：67.663　　総経過時間：393
500/2112個目完了　コメント数：19006　時間：70.199　　総経過時間：463
550/2112個目完了　コメント数：20138　時間：71.763　　総経過時間：535
600/2112個目完了　コメント数：22124　時間：82.197　　総経過時間：617
650/2112個目完了　コメント数：17805　時間：66.603　　総経過時間：684
700/2112個目完了　コメント数：18209　時間：73.130　　総経過時間：757
750/2112個目完了　コメント数：20201　時間：73.886　　総経過時間：831
800/2112個目完了　コメント数：27006　時間：101.250　　総経過時間：932
850/2112個目完了　コメント数：32692　時間：114.684　　総経過時間：1047
An error occurred: {'error': {'code': 403, 'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'errors': [{'m

In [58]:
# 前回の制限から再開
channel = "@quizknock" #@param{type:'string'}
file_path = f"data/{channel}/{channel}_videoIds.json"
video_ids = read_json(file_path).values[:,0].reshape(-1)
comments, error = get_comments(video_ids[985:], channel, k="1")

50/527個目完了　コメント数：75349　時間：240.435　　総経過時間：240
100/527個目完了　コメント数：75141　時間：245.891　　総経過時間：486
150/527個目完了　コメント数：63161　時間：208.592　　総経過時間：695
200/527個目完了　コメント数：43231　時間：145.818　　総経過時間：841
250/527個目完了　コメント数：42606　時間：137.839　　総経過時間：979
300/527個目完了　コメント数：38034　時間：124.157　　総経過時間：1103
350/527個目完了　コメント数：37078　時間：126.446　　総経過時間：1229
400/527個目完了　コメント数：24589　時間：86.642　　総経過時間：1316
450/527個目完了　コメント数：14290　時間：51.114　　総経過時間：1367
500/527個目完了　コメント数：9937　時間：36.843　　総経過時間：1404
527/527個目完了　コメント数：7721　時間：26.743　　総経過時間：1431


OSError: [Errno 28] No space left on device

In [17]:
# errorのビデオの確認
channel = "@kirin-k" #@param{type:'string'}
file_path = f"data/{channel}/{channel}_errorIds.json"
video_ids = read_json(file_path).values[:,0].reshape(-1)
comments, error = get_comments(video_ids, channel, k="1")

An error occurred: {'error': {'code': 403, 'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'errors': [{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]}}
video_id：XS95hsdgQOE　0
An error occurred: {'error': {'code': 403, 'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'errors': [{'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]}}
video_id：VHa3twLhCV0　1
An error occurred: {'error': {'code': 403, 'message': 'The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.', 'errors': [{'message': 'The

KeyboardInterrupt: 

In [52]:
# ファイルの結合
channel = "@kirin-k" #@param{type:'string'}
# 結合するJSONファイルのパス
file_paths = [f"data/{channel}/{channel}_comments.json", 
              f"data/{channel}/{channel}_comments1.json", 
              #f"data/{channel_name}/{channel_name}_comments2.json"
             ]

# 空のリストを作成して、各JSONファイルの内容を読み込み、結合する
merged_data = []
for file_path in file_paths:
    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)
        merged_data.extend(data)
        print(len(data))

# 結合したデータを新しいJSONファイルに保存
merged_file_path = f"data/{channel}/{channel}_comments_merged.json"
with open(merged_file_path, "w", encoding="utf-8") as file:
    json.dump(merged_data, file, ensure_ascii=False)
print(len(merged_data))

# ファイルの移動
destination_folder = "data/delete"
os.makedirs(destination_folder, exist_ok=True)
for file_path in file_paths:
    file_basename = os.path.basename(file_path)
    destination_path = os.path.join(destination_folder, file_basename)
    shutil.move(file_path, destination_path)

775072
1264
776336


In [50]:
def remove_duplicates_from_json(file_path, batch_size):
    unique_data = set()

    with open(file_path, "r", encoding="utf-8") as file:
        batch = []
        for line in file:
            data = json.loads(line)
            batch.append(data)

            if len(batch) >= batch_size:
                unique_data.update(batch)
                batch = []

        # 最後のバッチを処理
        unique_data.update(batch)

    unique_data = list(unique_data)

    with open(file_path, "w", encoding="utf-8") as file:
        for data in unique_data:
            file.write(json.dumps(data) + "\n")


        
file_name = "@nktofficial_comments"
# 重複を削除するJSONファイルのパス
file_path = f"data/comments/{file_name}_merged.json"

# 重複を削除
remove_duplicates_from_json(file_path, 10000)

MemoryError: 

In [28]:
"""
# ファイルの削除
for file_path in file_paths:
    os.remove(file_path)
"""